In [1]:
import pandas as pd
import json
from datetime import datetime
from pprint import pprint

In [333]:
ca_videos = pd.read_csv("youtube-new/CAvideos.csv")
de_videos = pd.read_csv("youtube-new/DEvideos.csv")
fr_videos = pd.read_csv("youtube-new/FRvideos.csv")
gb_videos = pd.read_csv("youtube-new/GBvideos.csv")
us_videos = pd.read_csv("youtube-new/USvideos.csv")

In [335]:
ca_videos.columns

Index([u'video_id', u'trending_date', u'title', u'channel_title',
       u'category_id', u'publish_time', u'tags', u'views', u'likes',
       u'dislikes', u'comment_count', u'thumbnail_link', u'comments_disabled',
       u'ratings_disabled', u'video_error_or_removed', u'description'],
      dtype='object')

In [336]:
len(ca_videos.trending_date.unique()) # YY-DD-MM

145

In [337]:
ca_videos.shape

(28919, 16)

In [338]:
def category_dict(file):
    category_dict = {}
    with open(file) as json_data:
        d = json.load(json_data)
        for item in d["items"]:
            category_dict[int(item["id"])] = item["snippet"]["title"]
    category_dict[29] = "Nonprofits & Activism"
    return category_dict

In [339]:
us_category = category_dict("youtube-new/US_category_id.json")

In [340]:
de_category = category_dict("youtube-new/DE_category_id.json")

In [341]:
ca_category = category_dict("youtube-new/CA_category_id.json")

In [342]:
gb_category = category_dict("youtube-new/GB_category_id.json")

In [343]:
fr_category = category_dict("youtube-new/FR_category_id.json")

In [344]:
ca_videos["category"] = ca_videos["category_id"].map(ca_category)

In [345]:
us_videos["category"] = us_videos["category_id"].map(us_category)

In [346]:
de_videos["category"] = de_videos["category_id"].map(de_category)

In [347]:
gb_videos["category"] = gb_videos["category_id"].map(gb_category)
fr_videos["category"] = fr_videos["category_id"].map(fr_category)

In [348]:
gb_videos["trending_date"] = gb_videos.trending_date.apply(lambda x: datetime.strptime(x, "%y.%d.%m"))

In [349]:
fr_videos = fr_videos.drop([18054, 18055], axis=0)

In [350]:
us_videos["trending_date"] = us_videos.trending_date.apply(lambda x: datetime.strptime(x, "%y.%d.%m"))
ca_videos["trending_date"] = ca_videos.trending_date.apply(lambda x: datetime.strptime(x, "%y.%d.%m"))

In [351]:
fr_videos["trending_date"] = fr_videos.trending_date.apply(lambda x: datetime.strptime(x, "%y.%d.%m"))
de_videos["trending_date"] = de_videos.trending_date.apply(lambda x: datetime.strptime(x, "%y.%d.%m"))

In [352]:
gb_videos["trending_month"] = gb_videos.trending_date.apply(lambda x: datetime.strftime(x, "%y-%m"))

In [353]:
us_videos["trending_month"] = us_videos.trending_date.apply(lambda x: datetime.strftime(x, "%y-%m"))

In [354]:
ca_videos["trending_month"] = ca_videos.trending_date.apply(lambda x: datetime.strftime(x, "%y-%m"))

In [355]:
fr_videos["trending_month"] = fr_videos.trending_date.apply(lambda x: datetime.strftime(x, "%y-%m"))

In [356]:
de_videos["trending_month"] = de_videos.trending_date.apply(lambda x: datetime.strftime(x, "%y-%m"))

### IF WE REMOVE ALL THE VIDEOS WITH 0 LIKES AND 0 COMMENTS

In [96]:
us_videos = us_videos[(us_videos.comment_count != 0) & (us_videos.likes != 0)]

#### dataframe for category counts

In [357]:
# def count_cat(df):
#     cat_count_df = df \
#         .groupby(["trending_month", "category"])["category_id"].count().reset_index(name="count")
#     return cat_count_df
def count_cat(df):
    cat_count_df = df \
        .groupby(["trending_month", "category"])\
        .agg({"category_id": "count", "views": "sum", "likes": "sum"})[["category_id", "views", "likes"]]\
        .reset_index()
    cat_count_df.columns = ["trending_month", "category", "cat_cnt", "sum_views_cnt", "sum_likes_cnt"]
    return cat_count_df

In [358]:
gb_mntly = count_cat(gb_videos)
gb_mntly["country"] = "Great Britain"

## --------------------- ##

In [359]:
us_mntly = count_cat(us_videos)
us_mntly["country"] = "United States"

## --------------------- ##

In [360]:
ca_mntly = count_cat(ca_videos)
ca_mntly["country"] = "Canada"

In [363]:
de_mntly = count_cat(de_videos)
de_mntly["country"] = "Germany"

In [106]:
gb_mntly.to_csv("gb_mntly.csv", index=False)

In [107]:
ca_mntly.to_csv("ca_mntly.csv", index=False)

In [108]:
us_mntly.to_csv("us_mntly.csv", index=False)

In [327]:
de_mntly.to_csv("de_mntly.csv", index=False)

## ---------------------------------

In [362]:
# de_mntly = pd.read_csv("de_mntly.csv")

In [364]:
de_mntly[de_mntly["trending_month"]=="18-01"]["sum_views_cnt"].nlargest(7)

35    814353108
39    749790520
46    240986849
33    220354912
42    210699546
36    175569694
37    128762516
Name: sum_views_cnt, dtype: int64

In [365]:
# us_mntly = us_mntly.drop([9], axis=0)

In [366]:
de_mntly.sum_views_cnt.describe()

count    9.700000e+01
mean     1.643050e+08
std      2.704849e+08
min      8.804000e+03
25%      1.331284e+07
50%      5.824060e+07
75%      2.106995e+08
max      1.667292e+09
Name: sum_views_cnt, dtype: float64

In [367]:
de_mntly.sum_likes_cnt.describe()

count    9.700000e+01
mean     5.940639e+06
std      1.042321e+07
min      0.000000e+00
25%      5.540400e+05
50%      1.619249e+06
75%      6.445181e+06
max      6.087613e+07
Name: sum_likes_cnt, dtype: float64

In [368]:
de_mntly = de_mntly.sort_values(by="sum_views_cnt", ascending=False).groupby("trending_month").head(5)

In [369]:
de_mntly.shape

(30, 6)

### combine more data

In [35]:
all_mntly = gb_mntly.append(fr_mntly).append(ca_mntly).append(us_mntly).append(de_mntly)

NameError: name 'gb_mntly' is not defined

### Plotly (us_mntly)

In [254]:
from plotly.offline import init_notebook_mode, iplot

import plotly
import plotly.plotly as py
from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as FF

init_notebook_mode(connected=True) 

In [255]:
import time
import pandas as pd

In [256]:
# us_mntly = pd.read_csv("us_mntly.csv")

In [257]:
# us_mntly["sum_comment_cnt"] = round((us_mntly["sum_comment_cnt"] / 1000), 3)

In [205]:
# us_mntly["sum_likes_cnt"] = round(us_mntly["sum_likes_cnt"] / 1000, 3)

In [371]:
plotly.tools.set_credentials_file(username='jingjuewang', api_key='Jlw5mC8COv1lXpqTUyMP')

In [372]:
table = FF.create_table(de_mntly.head(10))

In [373]:
iplot(table)

In [374]:
month_from_col = set(de_mntly["trending_month"])

In [375]:
month_list = sorted(list(month_from_col))

In [376]:
# make list of categories
categories = []
for category in de_mntly["category"]:
    if category not in categories:
        categories.append(category)

In [377]:
columns = []

In [378]:
# make grid
for month in month_list:
    for cat in categories:
        dataset_by_month = de_mntly[de_mntly["trending_month"] == month]
        dataset_by_month_and_cat = dataset_by_month[dataset_by_month["category"] == cat]
        for col_name in dataset_by_month_and_cat:
            column_name = '{month}_{cat}_{header}_gapminder_grid'.format(
                month=month, cat=cat, header=col_name
            )
            a_column = Column(list(dataset_by_month_and_cat[col_name]), column_name)
            columns.append(a_column)

In [380]:
# upload grid 
grid = Grid(columns)
url = py.grid_ops.upload(grid, 'de'+str(time.time()), auto_open=False)

In [17]:
url

'https://plot.ly/~jingjuewang/74/'

### make the figure

In [379]:
figure = {
    'data': [],
    "layout": {},
    "frames": [],
    "config": {'scrollzoom': True}
}

In [299]:
# fill in most of layout
# figure['layout']['xaxis'] = {'range': [1, 20000], 'title': 'total comment count', 'gridcolor': '#FFFFFF'}
# figure['layout']['yaxis'] = {'range': [8, 200000], 'title': 'total likes count', 'gridcolor': '#FFFFFF'}
# figure['layout']['hovermode'] = 'closest'
# figure['layout']['plot_bgcolor'] = 'rgb(223, 232, 243)'


In [381]:
x_min = de_mntly.sum_views_cnt.min()
x_max = de_mntly.sum_views_cnt.max()
y_min = de_mntly.cat_cnt.min()
y_max = de_mntly.cat_cnt.max()

In [382]:
# fill in most of layout
figure['layout']['xaxis'] = {'range': [x_min, x_max], "autorange": "true",'title': 'total views count', 'gridcolor': '#FFFFFF'}
figure['layout']['yaxis'] = {'range': [y_min, y_max], "autorange": "true",'title': 'category count', 'gridcolor': '#FFFFFF'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['plot_bgcolor'] = '#FFFFFF'

### Add slider

In [383]:
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'month:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

### Add Play and Pause Buttons

In [384]:
de_mntly.category.unique()

array([u'Entertainment', u'Music', u'People & Blogs', u'Sports',
       u'Film & Animation', u'Comedy'], dtype=object)

In [388]:
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 800, 'redraw': True},
                         'fromcurrent': True, 'transition': {'duration': 600, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

custom_colors = {
#     'Autos & Vehicles': 'rgb(171, 99, 250)',
    'Comedy': 'rgb(230, 99, 250)',
#     'Education': 'rgb(99, 110, 250)',
    'Entertainment': 'rgb(25, 211, 243)',
    'Film & Animation': 'rgb(50, 170, 255)',
#     'Gaming': 'rgb(188,128,189)',
#     'Howto & Style': 'rgb(141,211,199)',
    'Music': 'rgb(190,186,218)',
#     'News & Politics': 'rgb(251,128,114)',
    'People & Blogs': 'rgb(128,177,211)',
#     'Pets & Animals': 'rgb(253,180,98)',
#     'Science & Technology': 'rgb(179,222,105)',
    'Sports': 'rgb(171, 99, 250)'
#     'Travel & Events': 'rgb(217,217,217)',
#     'Nonprofits & Activism': 'rgb(188,128,189)',
#     'Shows': 'rgb(204,235,197)',
}

#### Fill in Figure with Data and Frames

In [389]:
col_name_template = '{month}_{cat}_{header}_gapminder_grid'
month = "17-11"
for cat in categories:
    data_dict = {
        'xsrc': grid.get_column_reference(col_name_template.format(
            month=month, cat=cat, header='sum_views_cnt'
        )),
        'ysrc': grid.get_column_reference(col_name_template.format(
            month=month, cat=cat, header='cat_cnt'
        )),
        'mode': 'markers',
        'textsrc': grid.get_column_reference(col_name_template.format(
            month=month, cat=cat, header='country'
        )),
        'marker': {
            'sizemode': 'area',
            'sizeref': 200000,
            'sizesrc': grid.get_column_reference(col_name_template.format(
                 month=month, cat=cat, header='sum_likes_cnt'
            )),
            'color': custom_colors[cat]
        },
        'name': cat
    }
    figure['data'].append(data_dict)

### create frames

In [390]:
for month in month_list:
    frame = {'data': [], 'name': str(month)}
    for cat in categories:
        data_dict = {
            'xsrc': grid.get_column_reference(col_name_template.format(
                month=month, cat=cat, header='sum_views_cnt'
            )),
            'ysrc': grid.get_column_reference(col_name_template.format(
                month=month, cat=cat, header='cat_cnt'
            )),
            'mode': 'markers',
            'textsrc': grid.get_column_reference(col_name_template.format(
                month=month, cat=cat, header='country'
                )),
            'marker': {
                'sizemode': 'area',
                'sizeref': 100000,
                'sizesrc': grid.get_column_reference(col_name_template.format(
                    month=month, cat=cat, header='sum_likes_cnt'
                )),
                'color': custom_colors[cat],
                "opacity": 0.8
            },
            'name': cat
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [month],
        {'frame': {'duration': 600, 'redraw': True},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': month,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

figure['layout']['sliders'] = [sliders_dict]

In [391]:
py.icreate_animations(figure, 'ca'+str(time.time()))